# Conceitos Básicos da Metodologia Gaia

- **Permissões:** Direitos que um agente tem sobre informações ou recursos, como ler (`reads`), modificar (`changes`) ou gerar (`generates`) dados.

- **Protocolos e Atividades:**  
  - *Atividades:* Ações internas realizadas pelo agente sem comunicação externa.  
  - *Protocolos:* Sequências de comunicação entre agentes para realizar tarefas conjuntas.

- **Responsabilidades:** Funções essenciais do agente, divididas em:  
  - *Liveness (Vivacidade):* Garantem que o agente continuará executando suas tarefas principais indefinidamente.  
  - *Safety (Segurança):* Restrições que garantem que o agente não provoque estados indesejados ou falhas no sistema.


---

## Sistema de Rede Inteligente Descentralizada

A Rede Inteligente Descentralizada é composta por múltiplos agentes que cooperam para equilibrar consumo, produção renovável e armazenamento em uma micro‑rede.

---

## Role Schema: HouseholdAgent (HA)

Description: Representa uma residência individual na micro‑rede.  
Cada HouseholdAgent consome energia conforme um perfil horário, pode produzir energia solar se for prosumer e gerencia uma bateria local, negociando com o GridNodeAgent para comprar ou vender energia dinamicamente.

Protocols and Activities:
- MonitorEnvironment → Recebe `environment_update` (irradiância, temperatura, vento) e atualiza a produção solar esperada.
- ComputeDemand → Calcula a demanda instantânea da residência para a hora simulada.
- ManageBattery → Controla carga e descarga da bateria local conforme excedente ou défice.
- RequestEnergy → Envia `energy_request` ao GNA quando a demanda é maior que a soma de produção local e bateria.
- RespondCFP → Participa de CFPs enviando `energy_offer` quando há excedente ou `declined_offer` quando decide não participar.
- SendStatusReport → Envia `status_report` periódico ao GNA com consumo, produção e estado da bateria.

Permissions:
- reads `solar_irradiance`, `temperature`, `wind_speed` → Para atualizar a estimativa de produção local.
- reads `battery_kwh`, `battery_capacity` → Para controlar o armazenamento local.
- reads `demand_profile` → Para determinar a demanda prevista por hora.
- reads `grid_price_signal` → Para reagir a sinais de preço embutidos nas CFPs e decisões de matching.
- generates `status_report` → Para informar seu estado energético ao GNA.
- generates `energy_request` → Para solicitar energia em caso de défice.
- generates `energy_offer` → Para oferecer excedentes de energia ao mercado.
- generates `declined_offer` → Para recusar participação em CFP quando não for atrativo.

Responsibilities – Liveness:
HouseholdAgent = (RoundCycle)^ω

RoundCycle =
    MonitorEnvironment
    → ComputeDemand
    → ManageBattery
    → (RequestEnergy | RespondCFP)
    → SendStatusReport

Responsibilities – Safety:
- `battery_kwh <= 0` -> `energy_offer = nil`          (não oferece energia se a bateria estiver vazia)
- `demand > production + battery` -> `RequestEnergy = initiate`  (solicita energia se a demanda exceder produção + bateria)
- `battery_kwh >= battery_capacity` -> `ManageBattery = stop_charging`  (interrompe carregamento quando a bateria está cheia)
- `not_prosumer = true` -> `energy_offer = nil`       (consumidor puro não se comporta como vendedor)

---

## Role Schema: StorageManagerAgent (SMA)

Description: Representa o sistema de armazenamento comunitário da micro‑rede.  
O StorageManagerAgent monitora o estado de carga da bateria comunitária, decide quando comprar energia para carregar e quando vender energia acumulada, podendo operar em modo emergencial quando o SOC está baixo.

Protocols and Activities:
- ReportBatteryStatus → Envia `statusBattery` periódico ao GNA com SOC e capacidade disponível.
- RespondCFP → Analisa CFPs e decide se envia `energy_offer` (venda) ou `energy_request` (compra).
- ExecuteControlCommand → Atualiza SOC em resposta a `control_command` e `offer_accept` do GNA.
- EmergencyLogic → Ativa modo de emergência quando SOC cai abaixo de um limiar, priorizando carga.

Permissions:
- reads `SOC`, `battery_capacity`, `mode_emergency` → Para decidir entre carga, descarga ou bloqueio de operação.
- reads CFP metadata (`round_id`, `deadline_ts`) → Para considerar prazos e contexto nas rodadas de leilão.
- generates `energy_offer` → Para oferecer energia comunitária ao mercado.
- generates `energy_request` → Para solicitar energia ao GNA para recarga.
- generates `statusBattery` → Para reportar o estado da bateria comunitária.
- generates respostas `offer_accept` → Para confirmar acordos envolvendo a bateria comunitária.

Responsibilities – Liveness:
StorageManagerAgent = (BatteryCycle)^ω

BatteryCycle =
    ReportBatteryStatus
    → RespondCFP
    → ExecuteControlCommand
    → EmergencyLogic

Responsibilities – Safety:
- `SOC <= emergency_threshold` -> `energyRequest = initiate`  (prioriza solicitar energia quando SOC está baixo)
- `SOC >= capacity`           -> `no_charge`                  (não aceita mais carga quando a bateria está cheia)
- `SOC <= 0`                  -> `no_discharge`               (não descarrega abaixo de zero)
- `mode_emergency = true`     -> `only_charge`                (em modo emergencial, bloqueia descarga)

---

## Role Schema: ProducerAgent (PA)

Description: Representa um produtor de energia renovável na rede, como unidades solares ou eólicas.  
O ProducerAgent calcula sua produção com base nas condições ambientais, incluindo ruído estocástico e falhas temporárias, oferecendo energia ao GridNodeAgent e reportando seu estado de produção.

Protocols and Activities:
- MonitorEnvironment → Recebe `environment_update` com irradiância, vento e temperatura.
- ComputeProduction → Calcula produção atual considerando capacidade instalada, clima, ruído e probabilidade de falha.
- SendProductionReport → Envia `production_report` ao GNA com a energia disponível.
- RespondCFP → Responde a CFPs com `energy_offer` ou `declined_offer` conforme produção e estado de falha.

Permissions:
- reads `solar_irradiance`, `wind_speed`, `temperature` → Para modelar a geração renovável.
- reads `production_capacity` → Para limitar a produção máxima.
- reads `failure_probability` → Para decidir falhas estocásticas.
- generates `energy_offer` → Para oferecer energia ao GNA.
- generates `production_report` → Para relatar geração em cada rodada.
- generates `declined_offer` → Para recusar participação em CFP quando não há energia disponível.

Responsibilities – Liveness:
ProducerAgent = (ProductionCycle)^ω

ProductionCycle =
    MonitorEnvironment
    → ComputeProduction
    → SendProductionReport
    → RespondCFP

Responsibilities – Safety:
- `production = 0`      -> `energy_offer = nil`  (não oferece energia quando a produção é nula)
- `failure = true`      -> `energy_offer = nil`  (em falha, não participa do mercado)
- `production < min_output` -> `offer = reduced` (oferta reduzida quando produção é muito baixa)

---

## Role Schema: GridNodeAgent (GNA)

Description: Atua como coordenador central da micro‑rede, equilibrando oferta e demanda de energia.  
O GridNodeAgent registra agentes, coleta relatórios, inicia rodadas de Contract Net, faz o matching entre ofertas e pedidos, aplica limites por agente e de transmissão e recorre à rede externa como fallback, registrando métricas em banco de dados.

Protocols and Activities:
- RegisterAgents → Realiza onboarding e registro de HAs, PAs, SMA e EA.
- CollectStatusReports → Recebe `status_report`, `production_report` e `statusBattery` a cada rodada.
- RequestEnvironmentUpdate → Solicita `environment_update` ao EnvironmentAgent.
- LaunchCFP → Envia CFP a agentes elegíveis como vendedores.
- MatchOffersRequests → Executa ordenação por preço, alocação parcial, limites por agente e limites de transmissão.
- ExternalGridFallback → Compra ou vende energia da/para a rede externa quando necessário.
- SendControlCommands → Envia `control_command` e `offer_accept` para aplicar alocações e atualizar estados de energia/SOC.
- RecordMetrics → Gera `performance_summary` e grava métricas e logs em SQLite.

Permissions:
- reads `status_report`, `production_report`, `statusBattery` → Para conhecer o estado global da micro‑rede.
- reads `energy_requests`, `energy_offers` → Para alimentar o mecanismo de matching.
- reads `environment_update` → Para contextualizar decisões de mercado.
- reads `grid_external_policy` → Para definir como usar a rede externa.
- generates `cfp` → Para iniciar rodadas do Contract Net.
- generates `control_command`, `offer_accept` → Para aplicar alocação de energia e atualizar SOC dos agentes.
- generates `environment_update` request → Para sincronizar o EA.
- generates `performance_summary` → Para logs e análise posterior.

Responsibilities – Liveness:
GridNodeAgent = (GridManagementCycle)^ω

GridManagementCycle =
    CollectStatusReports
    → RequestEnvironmentUpdate
    → LaunchCFP
    → MatchOffersRequests
    → ExternalGridFallback
    → SendControlCommands
    → RecordMetrics

Responsibilities – Safety:
- `seller_limit_exceeded = true`           -> `reduce_allocation`        (reduz alocação para o vendedor)
- `network_transmission_limit_exceeded = true` -> `stop_matching`        (interrompe novos matchs)
- `no_offers = true`                       -> `buy_from_external_grid`   (usa rede externa quando não há ofertas internas)
- `battery_full = true`                    -> `block_charge_commands`    (bloqueia comandos de carga para baterias cheias)
- `missing_reports = true`                 -> `mark_agent_inactive`      (marca agentes sem report como inativos)

---

## Role Schema: EnvironmentAgent (EA)

Description: Fornecedor de dados ambientais da simulação, gerando irradiância solar, vento e temperatura com base na hora simulada e no cenário configurado.  
O EnvironmentAgent atua sob demanda, respondendo a solicitações do GNA para garantir que todos os agentes usem um snapshot consistente do ambiente a cada rodada.

Protocols and Activities:
- ComputeWeather → Calcula `solar_irradiance`, `wind_speed` e `temperature` a partir de `current_hour` e `scenario_config`.
- BroadcastEnvironmentUpdate → Envia `environment_update` para os agentes registrados.

Permissions:
- reads `current_hour` → Para determinar o ponto no ciclo diário/sazonal.
- reads `scenario_config` → Para aplicar padrões climáticos definidos.
- generates `environment_update` → Para disponibilizar o estado climático atual aos demais agentes.

Responsibilities – Liveness:
EnvironmentAgent = (EnvironmentCycle)^ω

EnvironmentCycle =
    WaitForRequest
    → ComputeWeather
    → BroadcastEnvironmentUpdate

Responsibilities – Safety:
- `irradiance < 0`           -> `irradiance = 0`       (garante irradiância não negativa)
- `temperature < scenario_min` -> `temperature = scenario_min`
- `temperature > scenario_max` -> `temperature = scenario_max`
